In [ ]:
TO DO

general implementation
    create a graph or diagram to tell whats hapenning
    mark all the tasks/steps complete, incomplete, in progress, problems, to do, research etc.
    try on a toy problem

find new datasets
decide on splitting ratio 20 60 20 

try different similarity measures 
    reference paper
    cosine
    euclidean

In [ ]:
implementation steps

1. reading data and preprocessing
2. vectorization
    2.1 embeddings
    2.2 dimensionality reduction? (is similarity more accurate when dim. red. done) research
3. initial classifier to show results
4. calculate imbalance ratio and find the ratio of newly labeled data
5. oversample dataset using unlabeled set
    5.1 find the proper similarity function (eclidean, cosine etc.)
        Measurement of Text Similarity: A Survey: a very detailed survey of similarity functions that are used for text data
        https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html
        cosine similarity
        minkowski family (euclidean, manhattan)
        hamming distance
        Jaccard index
        Sorensen-dice index
        KL divergence
        Jensen–Shannon divergence with LDA
        Wasserstein distance
        SMTP 
        word mover’s distance
    5.2 define a threshold or mechanism to add data for multilabeled set
6. train a final classifier to compare results

In [2]:
import pandas as pd
import numpy as np
import preprocess

In [ ]:
def read_data(path):
    df = pd.read_csv(path)
    df['text'] = df['text'].apply(preprocess.preprocess_text)
    return df

In [ ]:
def vectorize_data(df, model_name='stsb-roberta-large'):
    from sentence_transformers import util
    from sentence_transformers import SentenceTransformer
    import torch
    
    model = SentenceTransformer(model_name)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    model.encode(text1, convert_to_tensor=False, device=device)

In [3]:
majority_path = r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv'
df = pd.read_csv(majority_path)

In [ ]:
def classifier(X, y):
    pass

In [ ]:
def find_balancing_ratio(y):
    balancing_ratio = 1.0
    return balancing_ratio

In [ ]:
def oversample_dataset(labeled_data, unlabeled_data):
    return labeled_data, unlabeled_data

In [ ]:
X = df['text']
y = df.drop(['text'], axis=1)